In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [3]:
# Mengatur parameter
batch_size = 32
img_height = 180
img_width = 180

In [4]:
# Mengatur direktori dataset
train_dir = '../data/train'
val_dir = '../data/validation'

# Cek apakah direktori ada
if not os.path.exists(train_dir):
    print(f"Direktori train tidak ditemukan: {train_dir}")
if not os.path.exists(val_dir):
    print(f"Direktori validation tidak ditemukan: {val_dir}")

In [5]:
# Menggunakan ImageDataGenerator untuk augmentasi data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Mengalirkan data dari direktori
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 2851 images belonging to 26 classes.
Found 2851 images belonging to 26 classes.


In [7]:
# Membangun model CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Jumlah kelas
])

# Mengompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

c:\HASTRA_VANI\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
@tf.function
def train_step_fn(image_list, groundtruth_boxes_list, groundtruth_classes_list, model, optimizer, vars_to_fine_tune):
    """A single training iteration."""
    with tf.GradientTape() as tape:
        # Pastikan image_list adalah tensor 4-D
        preprocessed_images = tf.convert_to_tensor(image_list)  # Langsung menggunakan tensor

        # Make a prediction
        prediction_dict = model(preprocessed_images, training=True)

        # Gunakan tf.print untuk memeriksa tipe dan bentuk prediction_dict
        tf.print("Type of prediction_dict:", type(prediction_dict))
        tf.print("Shape of prediction_dict:", tf.shape(prediction_dict))

        # Jika prediction_dict adalah tensor, cetak bentuknya
        if isinstance(prediction_dict, tf.Tensor):
            tf.print("prediction_dict is a tensor with shape:", prediction_dict.shape)
            # Jika output adalah tensor, Anda mungkin perlu memprosesnya secara berbeda
            predictions = prediction_dict  # Sesuaikan ini sesuai kebutuhan
        elif isinstance(prediction_dict, dict):
            tf.print("Keys in prediction_dict:", prediction_dict.keys())
            predictions = prediction_dict.get('class_predictions_with_background', None)  # Ganti dengan kunci yang benar
            if predictions is None:
                tf.print("Warning: 'class_predictions_with_background' not found in prediction_dict.")
                return None  # Atau tangani kesalahan sesuai kebutuhan
        else:
            tf.print("prediction_dict is of an unexpected type.")
            return None  # Atau tangani kesalahan sesuai kebutuhan

        # Flatten predictions
        predictions = tf.reshape(predictions, (-1, 26))  # Pastikan dimensi sesuai

        # Ubah bentuk groundtruth_classes_list menjadi one-hot encoding
        groundtruth_classes_one_hot = tf.one_hot(groundtruth_classes_list, depth=26)  # Ubah sesuai jumlah kelas
        groundtruth_classes_one_hot = tf.reshape(groundtruth_classes_one_hot, (-1, 26))  # Pastikan bentuknya sesuai

        # Calculate the total loss (sum of both losses)
        loss_value = tf.keras.losses.categorical_crossentropy(
            groundtruth_classes_one_hot,  # Gunakan one-hot encoded classes
            predictions,
            from_logits=True
        )
        total_loss = tf.reduce_mean(loss_value)

        # Calculate the gradients
        gradients = tape.gradient(total_loss, vars_to_fine_tune)

        # Optimize the model's selected variables
        optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))

    return total_loss

print(f"Image tensors shape: {[img.shape for img in image_tensors]}")

Image tensors shape: []


In [10]:
# Menjalankan pelatihan
num_batches = 100  # Misalnya, Anda ingin menjalankan 100 batch
optimizer = tf.keras.optimizers.Adam()
to_fine_tune = model.trainable_variables

print('Start fine-tuning! ', flush=True)

for idx in range(num_batches):
    # Ambil kunci untuk subset contoh secara acak
    all_keys = list(range(len(train_generator)))
    np.random.shuffle(all_keys)
    example_keys = all_keys[:batch_size]

    # Mendapatkan ground truth
    gt_boxes_list = [None] * batch_size  # Ganti dengan ground truth bounding boxes jika ada
    gt_classes_list = [train_generator.classes[key] for key in example_keys]  # Mengambil kelas dari generator

    # Mendapatkan gambar
    image_tensors = [train_generator[key][0] for key in example_keys]

    # Langkah pelatihan (forward pass + backward pass)
    total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list, model, optimizer, to_fine_tune)

    if idx % 10 == 0:
        print('batch ' + str(idx) + ' of ' + str(num_batches) + ', loss=' + str(total_loss.numpy()), flush=True)

print('Done fine-tuning!')

Start fine-tuning! 


ValueError: in user code:

    File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_38408\3647462354.py", line 9, in train_step_fn  *
        prediction_dict = model(preprocessed_images, training=True)
    File "c:\HASTRA_VANI\myenv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "c:\HASTRA_VANI\myenv\Lib\site-packages\keras\src\models\functional.py", line 273, in _adjust_input_rank
        raise ValueError(

    ValueError: Exception encountered when calling Sequential.call().
    
    [1mInvalid input shape for input Tensor("packed:0", shape=(32, 32, 180, 180, 3), dtype=float32). Expected shape (None, 180, 180, 3), but input has incompatible shape (32, 32, 180, 180, 3)[0m
    
    Arguments received by Sequential.call():
      • inputs=tf.Tensor(shape=(32, 32, 180, 180, 3), dtype=float32)
      • training=True
      • mask=None
